In [10]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation,Input, Merge, add
from keras.layers.normalization import BatchNormalization
import keras.optimizers
import numpy as np
import pandas as pd
import random

In [2]:
data = pd.read_csv('../Data/trainset_1120.csv')
x = data.drop(data.columns[[0, 1]], axis=1)
x = np.array(x)
y = data['target']
y = np.array(y)

In [3]:
1-np.mean(y)

0.96355248214081701

In [4]:
random.seed(867)
train_index = np.random.choice(range(data.shape[0]),int(0.8*data.shape[0]))
validate_index=set(range(data.shape[0]))-set(train_index)
validate_index=np.array(list(validate_index),dtype='int64')

In [5]:
x_train = x[train_index,:]
x_validate = x[validate_index,:]
y_train = y[train_index]
y_validate = y[validate_index]

In [8]:
case_index = [i for i,t in enumerate(y_train) if t == 1]
control_index = [i for i,t in enumerate(y_train) if t == 0]

In [9]:
new_control_index = np.random.choice(control_index,4*len(case_index))
new_index = np.concatenate((new_control_index,case_index))
new_index

array([401579, 393867,   8966, ..., 476123, 476143, 476167])

In [11]:
new_x_train = x_train[new_index,:]
new_y_train = y_train[new_index]

In [100]:
### test tensorflow, don't run
import keras.backend as K
import tensorflow as tf
a=np.array([1,0,0,0,1,0])
p=np.array([0,0,0,1,1,0])
actual = K.variable(value=a)
pred = K.variable(value=p)
nT = K.shape(actual)[-1]
n = K.cast(nT, dtype='int32')
n_float = K.cast(nT, dtype=K.floatx())
inds = tf.nn.top_k(pred, n)[1]
a_s = K.gather(actual, inds)
a_c = K.cumsum(a_s)
s1 = K.sum(a_c)
s2 = K.sum(a_s)
giniSum = K.cast(tf.divide(s1,s2),dtype=K.floatx()) - K.cast(tf.divide(n+1,2),dtype=K.floatx())
standard_gini = K.cast(tf.divide(giniSum,n_float),dtype=K.floatx())
K.eval(standard_gini)

0.16666667

In [14]:
import keras.backend as K
import tensorflow as tf
def gini(actual, pred):
    nT = K.shape(pred)[0]
    n = K.cast(nT, dtype='int32')
    n_float = K.cast(nT, dtype=K.floatx())
    actual = K.reshape(actual,(1,n))[-1]
    pred = K.reshape(pred,(1,n))[-1]
    inds = tf.nn.top_k(pred, n)[1]
    a_s = K.gather(actual, inds)
    a_c = K.cumsum(a_s)
    s1 = K.sum(a_c)
    s2 = K.sum(a_s)
    giniSum = K.cast(tf.divide(s1,s2),dtype=K.floatx()) - K.cast(tf.divide(n+1,2),dtype=K.floatx())
    standard_gini = K.cast(tf.divide(giniSum,n_float),dtype=K.floatx())
    return standard_gini

def gini_normalized(a, p):
    norm_gini = K.cast(tf.divide(gini(a, p),gini(a, a)),dtype=K.floatx())
    return norm_gini

## new loss
## pred  actual  loss
##  0       1      49    
##  1       0      1 
##  1/0     1/0    0
##  a=25,b=-24,c=0
##  if all 0, loss=0.05*9
##  if all 1, loss=0.95

def new_loss(actual,pred):
    return K.mean(25*K.square(pred-actual)-24*(pred-actual),axis=-1)

In [16]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=x_train.shape[1]))
model.add(Dropout(0.3))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
sgd = keras.optimizers.Adagrad(lr=0.01, epsilon=1e-08, decay=0.01)
model.compile(loss=new_loss, optimizer=sgd, metrics=[gini_normalized])
model.fit(x_train, y_train, epochs=20, batch_size=4096, class_weight={0:1,1:26.6})

Epoch 1/20
476169/476169 [==============================] - 9s - loss: 1.0945 - gini_normalized: 0.0019     
Epoch 2/20
476169/476169 [==============================] - 8s - loss: 0.9658 - gini_normalized: 0.0095     
Epoch 3/20
476169/476169 [==============================] - 8s - loss: 0.9703 - gini_normalized: -0.0026     
Epoch 4/20
208896/476169 [============>.................] - ETA: 5s - loss: 0.9631 - gini_normalized: -0.0122

KeyboardInterrupt: 